In [1]:
!pip install jieba

In [2]:
!pip install gensim

In [3]:
!mkdir jieba_data

mkdir: cannot create directory ‘jieba_data’: File exists


In [4]:
!wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big -O jieba_data/dict.txt.big

--2020-05-31 16:48:43--  https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big [following]
--2020-05-31 16:48:44--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.108.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [text/plain]
Saving to: ‘jieba_data/dict.txt.big’

jieba_data/dict.txt 100%[===================>]   8.18M  --.-KB/s    in 0.07s   

2020-05-31 16:48:46 (115 MB/s) - ‘jieba_data/dict.txt.big’ saved [8583143/8583143]



### 載入 jieba 函式庫

In [5]:
import jieba

### 載入繁體中文詞庫

In [6]:
jieba.set_dictionary('jieba_data/dict.txt.big')

### 開啟文字檔

In [7]:
# 開檔
fileAllLines = []
with open('Three Kingdoms.txt') as fileLine:
    for line in fileLine:
        #print(line)
        fileAllLines.append(line)

### 進行斷字斷詞

In [8]:
# 斷詞
seg = []
for i in range(len(fileAllLines)):
    #cut_list = list( jieba.cut_for_search(fileAllLines[i]) )
    cut_list = list( jieba.cut(fileAllLines[i], cut_all = False) )
    seg.append([' '.join([ item for item in cut_list if len(item) > 1 ])])

Building prefix dict from /home/jovyan/work/jieba_data/dict.txt.big ...
Dumping model to file cache /tmp/jieba.ud3c40470d3518910bfd0dc86d865fad4.cache
Loading model cost 1.937 seconds.
Prefix dict has been built successfully.


In [9]:
print(len(seg))

6001


In [10]:
seg = [ s for s in seg if len(s) > 0] 

### 準備存檔，將斷好的字詞存下來。先刪除之前留下的紀錄檔案(segDone.txt)

In [11]:
!rm segDone.txt

### 斷詞結果存檔

In [12]:
# 斷詞結果存檔
segSaveFile = 'segDone.txt'
with open(segSaveFile, 'wb') as saveFile:
    for i in range(len(seg)):
        words = seg[i][0].encode('utf-8')
        words = words.strip()
        if len(words) > 0:
            #saveFile.write(seg[i][0].encode('utf-8'))
            saveFile.write(words)
            saveFile.write('\n'.encode())

In [13]:
!head -10 segDone.txt

三國演義 作者 羅貫中
簡介
三國演義 一本 長篇 歷史小說 可以 中國 古代 長篇 章回小說 開山 之作 四大名著 之一 作者 明朝 羅貫中 故事 黃巾起義 終於 西晉 統一 百年 萬事 七實 三虛 三國 小說 通篇 精巧 敘述 謀略 被譽 中國 謀略 全書
羅貫中 1330 1400 之間 名本 湖海 散人 明代 通俗小說 籍貫 一說 太原 山西 一說 錢塘 浙江 杭州 不可 確考 據傳說 羅貫中 充任 過元末 農民 起義軍 張士誠 幕客 三國 通俗 演義 他還 創作 隋唐 志傳 通俗小說 太祖 龍虎風雲 戲劇 另外 相當 一部分 認為 水滸傳 三十 所作
目錄
001 桃園 豪傑 結義 黃巾 英雄 立功 002 張翼德 鞭督郵 何國舅
003 議溫明 董卓 叱丁原 饋金珠 李肅 呂布 004 廢漢帝 踐位 董賊 孟德獻刀
005 矯詔 諸鎮應 曹公 破關 兵三英 戰呂布 006 焚金 董卓 行兇 玉璽 孫堅 背約
007 袁紹 河戰 公孫 孫堅 跨江擊 劉表 008 王司徒 巧使 連環計 太師 大鬧 鳳儀亭
009 除暴 呂布 司徒 長安 賈詡 010 王室 騰舉義 報父仇 曹操 興師


In [14]:
!cat segDone.txt | wc -c

1314940


### 載入word2vec 函式庫
### 參考網頁：https://radimrehurek.com/gensim/models/word2vec.html

In [15]:
from gensim.models import word2vec

### 一行一行的從檔案中取出句子：一行一個句子，句子中的字必須預先處理好並以空白隔開

In [16]:
sentences = word2vec.LineSentence("segDone.txt")

In [17]:
import time

### 訓練模型
#### size: 向量維度 = 300
#### iter: 迭代次數 = 100
#### sg: 0(CBOW), 1(Skip-gram)

In [18]:
start_time = time.time()
model = word2vec.Word2Vec(sentences, size=300, iter=100, sg=0, window=15, workers=3)
print("--- spend %s seconds ---" % (time.time() - start_time))

--- spend 24.617108583450317 seconds ---


In [19]:
model

### 儲存訓練好的word2vec 模型

In [20]:
model.save("word2vec.model")

In [21]:
model.corpus_total_words

175124

# 詞相似度

In [22]:
model.wv.most_similar('赤壁')

[('赤壁鏖兵', 0.4455590844154358),
 ('我心', 0.4383259117603302),
 ('東風', 0.42348527908325195),
 ('干戈', 0.4116250276565552),
 ('吳有', 0.40699321031570435),
 ('動干戈', 0.40697914361953735),
 ('顧雍', 0.4059343934059143),
 ('謀臣', 0.3931702971458435),
 ('有意', 0.3872483968734741),
 ('孫仲謀', 0.38655227422714233)]

In [23]:
model.wv.similar_by_word('赤壁')

[('赤壁鏖兵', 0.4455590844154358),
 ('我心', 0.4383259117603302),
 ('東風', 0.42348527908325195),
 ('干戈', 0.4116250276565552),
 ('吳有', 0.40699321031570435),
 ('動干戈', 0.40697914361953735),
 ('顧雍', 0.4059343934059143),
 ('謀臣', 0.3931702971458435),
 ('有意', 0.3872483968734741),
 ('孫仲謀', 0.38655227422714233)]

In [24]:
model.wv.similarity('曹操','呂布')

0.06339327

In [25]:
model.wv.similar_by_word('曹操')

[('相持', 0.3405546247959137),
 ('謀士', 0.325483500957489),
 ('操聞', 0.3154907524585724),
 ('許昌', 0.31446385383605957),
 ('荀彧', 0.30381664633750916),
 ('南征', 0.3038089871406555),
 ('許都', 0.2955462336540222),
 ('劉豫州', 0.29119184613227844),
 ('馬頭', 0.28970250487327576),
 ('地面', 0.28432604670524597)]

In [26]:
model.wv.similar_by_word('呂布')

[('侯成', 0.5209932923316956),
 ('宋憲', 0.5085991024971008),
 ('高順', 0.5057452917098999),
 ('魏續', 0.4937801957130432),
 ('呂布乃', 0.4397823214530945),
 ('布乃', 0.43686139583587646),
 ('田氏', 0.42838144302368164),
 ('紀靈', 0.4246787428855896),
 ('濮陽', 0.42379873991012573),
 ('臧霸', 0.3928311765193939)]

In [27]:
model.wv.similarity('孔明','諸葛亮')

0.16280296

In [28]:
model.wv.similar_by_word('孔明')

[('孔明自', 0.48412495851516724),
 ('三洞', 0.3980969786643982),
 ('孔明問', 0.36327099800109863),
 ('孔明之', 0.35996299982070923),
 ('羽扇', 0.35514408349990845),
 ('孔明遂', 0.34788239002227783),
 ('金環三結', 0.3424835801124573),
 ('孔明笑', 0.341704785823822),
 ('圖本', 0.34110069274902344),
 ('魏延之', 0.33825427293777466)]

In [29]:
model.wv.similar_by_word('諸葛亮')

[('子丹', 0.3594045639038086),
 ('兵符', 0.35746103525161743),
 ('深通', 0.3559360206127167),
 ('奏聞', 0.3433877229690552),
 ('成大功', 0.340515673160553),
 ('武鄉侯', 0.32972708344459534),
 ('靳祥', 0.323337197303772),
 ('細言', 0.3216300308704376),
 ('仲達', 0.3184889256954193),
 ('兵法', 0.31379878520965576)]

In [30]:
model.wv.similarity('孔明','周瑜')

0.08637422

In [31]:
model.wv.similarity('孔明','司馬懿')

0.26184645

In [33]:
input_string = input('關鍵字:')
model.wv.similar_by_word(input_string)

關鍵字:關羽


[('弓手', 0.5248792767524719),
 ('天人', 0.5225162506103516),
 ('此心', 0.5147749185562134),
 ('結為', 0.50413978099823),
 ('桃園', 0.4986562132835388),
 ('冒瀆', 0.4268714487552643),
 ('名爵', 0.42649543285369873),
 ('操以', 0.42396464943885803),
 ('可謂', 0.4205969572067261),
 ('魏延為', 0.41861289739608765)]

# 詞預測

In [35]:
result = model.predict_output_word('關羽')
print(result)

None


# 類推 (analogy)

#### 基本範例 (https://radimrehurek.com/gensim/models/keyedvectors.html)

In [36]:
import gensim.downloader as api

In [37]:
word_vectors = api.load("glove-wiki-gigaword-100")  # load pre-trained word-vectors from gensim-data

In [38]:
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [39]:
result = word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.8965


In [40]:
ans1 = model.wv.most_similar(positive=['劉備','孔明'], negative=['孫權'])
print("%s: %s"%(ans1[0]))

王忠: 0.28848162293434143


In [41]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

In [42]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

In [43]:
print(x_vals)
print(y_vals)
print(labels)

[18.723291, 22.686256, -4.320035, 25.229263, 14.772188, 29.247107, 14.581709, -3.24282, -23.638752, 43.989468, 25.477697, -24.01008, 44.28435, 31.945602, 44.277702, 1.3103527, 32.575848, 35.419975, -0.9029668, 24.571312, 40.459076, -27.07599, 46.55591, 6.8255506, 48.53684, -6.139895, 15.734907, 36.576447, 11.40858, 53.106876, 13.206782, 43.870857, 5.6436462, 21.33909, 56.63148, 19.04841, 20.899273, -14.380032, -20.04946, 24.207935, 13.6655655, 43.592064, 25.421684, 23.685047, 25.886524, 25.869293, -8.471478, 34.646877, 26.435057, -4.030406, -18.726978, 1.1394626, 0.34939966, 12.141679, -11.438782, 2.2067957, 2.1030338, -14.509993, -10.850144, 5.0756454, 34.077682, -21.727718, -12.010236, -17.60893, -12.76632, 4.996655, 6.099103, 40.23898, 13.250479, 26.548714, 1.7695799, -13.95096, -24.867405, 5.5097556, 26.894918, 4.3999634, 38.89391, -10.664622, -26.326857, 32.652687, -30.257074, -19.72418, 51.59926, -33.485485, -17.042309, 22.023829, 36.397858, 4.100533, 35.765312, 21.652487, 29.782

In [44]:
!pip install plotly

# 模型視覺化

In [45]:
def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)